# Model

This model is adopted from an open-source autonomous robot project. It's a pretty simple CNN model. First several conv layers are 5x5 kernel with stride 2x2.

The model has 2 output to represent 2 classes: *good* and *bad*. The activation is softmax. Loss function is cross entrophy.

In [3]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROnPlateau

img_width, img_height = 224,224
train_data_dir = "/data/generated/train-all-videos"
validation_data_dir = "/data/generated/val-all-videos"
batch_size = 32
epochs = 1000

### Build the network 
img_input = Input(shape=(img_width, img_height, 3))
x = Conv2D(24, (5,5), strides=(2,2), activation='relu')(img_input)
x = Conv2D(32, (5,5), strides=(2,2), activation='relu')(x)
x = Conv2D(64, (5,5), strides=(2,2), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Conv2D(64, (3,3), strides=(1,1), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Conv2D(128, (3,3), strides=(1,1), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = img_input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 110, 24)      1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 53, 53, 32)        19232     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
__________

# Training

The entire data set are split into 2 parts by *videos*:

- Test set: contains 14 videos. All these 14 videos start with good print but fail at the end. These videos are then sliced into still images. ~10k "good" images and ~4k "bad" images

- Training + Validation set: Contains ~100 videos of "failed" jobs and 10k+ videos of "good" jobs. The good vidoes are heavily down-sampled to make the data set a bit more balanced. Then these videos are sliced into still images. ~210k images labelled "good" and ~30k images labelled "bad". This set is then randomly split (75/25) into training and validation sets.

This notebook shows the traing with 1:5 weighted loss to account for the unbalanced data set.

This model is somehow different from any other models in that the validation loss kept on going down for 200+ epochs (7 days of training) until it reached about `0.05`. Most other models I have trained so far can only go as low as `0.2`.

Note: When the model is trained with unweighted loss (not shown here), the model will end up with predicting most of the images as "good" (since they account for 90% of the samples).

In [ ]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
vertical_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

val_datagen = ImageDataGenerator(
rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size,
shuffle=True,
class_mode = "categorical")

validation_generator = val_datagen.flow_from_directory(
validation_data_dir,
batch_size = batch_size,
target_size = (img_height, img_width),
shuffle=True,
class_mode = "categorical")

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("small_model.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')
plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)


Found 180537 images belonging to 2 classes.
Found 58489 images belonging to 2 classes.


In [ ]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
#     steps_per_epoch=200,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
#     validation_steps=50,
    class_weight = {0:1 , 1:5},
    callbacks = [checkpoint, early])

Epoch 1/1000
5641/5641 [==============================] - 2075s 368ms/step - loss: 0.9891 - acc: 0.8748 - val_loss: 0.5632 - val_acc: 0.8818

Epoch 00001: val_loss improved from inf to 0.56321, saving model to small_model.h5
Epoch 2/1000
5641/5641 [==============================] - 2087s 370ms/step - loss: 0.9804 - acc: 0.8745 - val_loss: 0.5640 - val_acc: 0.8816

Epoch 00002: val_loss did not improve
Epoch 3/1000
5641/5641 [==============================] - 2084s 369ms/step - loss: 0.9743 - acc: 0.8565 - val_loss: 0.5697 - val_acc: 0.8808

Epoch 00003: val_loss did not improve
Epoch 4/1000
5641/5641 [==============================] - 2076s 368ms/step - loss: 0.9693 - acc: 0.8404 - val_loss: 0.5678 - val_acc: 0.8807

Epoch 00004: val_loss did not improve
Epoch 5/1000
5641/5641 [==============================] - 2086s 370ms/step - loss: 0.9639 - acc: 0.8327 - val_loss: 0.5347 - val_acc: 0.8822

Epoch 00005: val_loss improved from 0.56321 to 0.53469, saving model to small_model.h5
Epoch 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4554/5641 [=======================>......] - ETA: 6:24 - loss: 0.6624 - acc: 0.8680

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5641/5641 [==============================] - 2069s 367ms/step - loss: 0.6494 - acc: 0.8738 - val_loss: 0.4969 - val_acc: 0.7643

Epoch 00020: val_loss did not improve
Epoch 21/1000
 459/5641 [=>............................] - ETA: 29:55 - loss: 0.6473 - acc: 0.8711

...

5641/5641 [==============================] - 2077s 368ms/step - loss: 0.1752 - acc: 0.9614 - val_loss: 0.0784 - val_acc: 0.9686

Epoch 00247: val_loss did not improve
Epoch 248/1000
 272/5641 [>.............................] - ETA: 30:34 - loss: 0.1646 - acc: 0.9627

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5641/5641 [==============================] - 2068s 367ms/step - loss: 0.1705 - acc: 0.9614 - val_loss: 0.0699 - val_acc: 0.9717

Epoch 00248: val_loss did not improve
Epoch 249/1000
4187/5641 [=====================>........] - ETA: 8:34 - loss: 0.1727 - acc: 0.9627

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5641/5641 [==============================] - 2069s 367ms/step - loss: 0.1718 - acc: 0.9627 - val_loss: 0.0662 - val_acc: 0.9731

Epoch 00249: val_loss did not improve
Epoch 250/1000
5641/5641 [==============================] - 2078s 368ms/step - loss: 0.1720 - acc: 0.9620 - val_loss: 0.0635 - val_acc: 0.9745

Epoch 00250: val_loss did not improve
Epoch 251/1000
5641/5641 [==============================] - 2070s 367ms/step - loss: 0.1711 - acc: 0.9623 - val_loss: 0.0948 - val_acc: 0.9630

Epoch 00251: val_loss did not improve
Epoch 252/1000
5641/5641 [==============================] - 2072s 367ms/step - loss: 0.1729 - acc: 0.9615 - val_loss: 0.0625 - val_acc: 0.9742

Epoch 00252: val_loss did not improve
Epoch 253/1000
5641/5641 [==============================] - 2064s 366ms/step - loss: 0.1742 - acc: 0.9616 - val_loss: 0.0846 - val_acc: 0.9641

Epoch 00253: val_loss did not improve
Epoch 254/1000
5641/5641 [==============================] - 2072s 367ms/step - loss: 0.1759 - acc: 0.9611

# Test if the model generalizes to videos not seen before

In [4]:
import keras
import os
import numpy as np
import pandas as pd
from PIL import Image
import glob
from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
import matplotlib.pyplot as plt

model_name = 'small_model.h5'

model = keras.models.load_model(model_name)
model.summary()

def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="float32" )/255.0
    return data

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 110, 110, 24)      1824      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 53, 53, 32)        19232     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 64)        51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
__________

### `test-224` contains videos that are not part of training or validation set.

When the model is evaluated on the test set, it exhibits much higher loss (`2.5`) and lower accuracy (`0.69`) than validation loss/accuracy.

My hypothesis for the overfitting is that the model learned the "wrong features", because the still images in validation and training sets share too much similarity since they came from the same set of videos. 

In [12]:
test_datagen = ImageDataGenerator(
rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
'/data/generated/test-224/',
batch_size = batch_size,
target_size = (img_height, img_width),
shuffle=True,
class_mode = "categorical")
(loss, acc) = model.evaluate_generator(test_generator, steps=50, workers=2, use_multiprocessing=True)
print("Loss:" + str(loss))
print("Accuracy:" + str(acc))

Found 13420 images belonging to 2 classes.
Loss:2.509437069892883
Accuracy:0.69625
